<a href="https://colab.research.google.com/github/Luca-Skyline/Check4Mate/blob/main/Chess_Piece_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import pickle
import gc

print(tf.__version__)

2.13.0


In [2]:
!git clone https://github.com/Luca-Skyline/check4mate/
%cd /content/check4mate/Chess_Classification_Dataset

fatal: destination path 'check4mate' already exists and is not an empty directory.
/content/check4mate/Chess_Classification_Dataset


In [ ]:
pickled_train_images = []
train_labels = []

for class_index, folder in enumerate(os.listdir('train/')):
  train_images = []
  current_folder_path = os.path.join('train/', folder)
  if os.path.isdir(current_folder_path):
    for image in os.listdir(current_folder_path):
      current_image_path = os.path.join(current_folder_path, image)
      if os.path.isfile(current_image_path):
        image = cv2.imread(current_image_path, cv2.IMREAD_GRAYSCALE)
        pixel_array = np.array(image.tolist())
        train_images.append(pixel_array)
        train_labels.append(class_index)
  train_images = np.array(train_images)
  pickled_train_images.append(pickle.dumps(train_images))
  # with open(f'trainImages{class_index}.txt', 'wb') as f1:
  #   pickle.dump(train_images, f1)

train_labels = np.array(train_labels)

pickled_train_labels = pickle.dumps(train_labels)

del train_labels
gc.collect()

In [ ]:
test_images = []
test_labels = []

for class_index, folder in enumerate(os.listdir('valid/')):
  current_folder_path = os.path.join('valid/', folder)
  if os.path.isdir(current_folder_path):
    for image in os.listdir(current_folder_path):
      current_image_path = os.path.join(current_folder_path, image)
      if os.path.isfile(current_image_path):
        image = cv2.imread(current_image_path, cv2.IMREAD_GRAYSCALE)
        pixel_array = np.array(image.tolist())
        test_images.append(pixel_array)
        test_labels.append(class_index)

test_images = np.array(test_images)
test_labels = np.array(test_labels)

pickled_test_images = pickle.dumps(test_images)
pickled_test_labels = pickle.dumps(test_labels)

# with open('testImages.txt', 'wb') as f1:
#    pickle.dump(test_images, f1)

# with open('testLabels.txt', 'wb') as f1:
#    pickle.dump(test_labels, f1)

In [ ]:
class_names = ['bb', 'bk', 'bn', 'bp', 'bq',
               'br', 'empty', 'wb', 'wk', 'wn',
               'wp', 'wq', 'wr']

for i in range(10):
  pickled_train_images[i] = pickle.dumps(pickle.loads(pickled_train_images[i]) / 255.0)

pickled_test_images = pickle.dumps(pickle.loads(pickled_test_images) / 255.0)

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(pickle.loads(pickled_train_images[0])[i])
    # The CIFAR labels happen to be arrays,
    # which is why you need the extra index
    plt.xlabel(class_names[pickle.loads(pickled_train_labels)[i]])
plt.show()

In [ ]:
dataset.location

'/content/Chess-Piece-Detector-1'